# ***Google Drive Bootup***

In [1]:
import os.path
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from httplib2 import Http
from httplib2 import Http
from apiclient.http import MediaFileUpload,MediaIoBaseDownload
import io

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/drive.metadata']

creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret_833249459653-01pcjla0c9re30fgr2dfipcgg5lgq6j0.apps.googleusercontent.com.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
    
# Call the Drive v3 API
drive = build("drive", "v3", credentials=creds)

# ***Downloads ALL Folders***

### ***Verifies and Reports Files and Folders***

In [2]:
FolderName = 'ALL'
PC = 14

query = "mimeType = 'application/vnd.google-apps.folder'"

from time import mktime
import datetime
from datetime import datetime
import os.path
import pandas as pd
import numpy as np

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os
import subprocess
import time
global dffilecount, filename, foldersize, folderid, FolderName2

currentDateTime = datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y") #except: 2022
date.strftime("%Y")

d = datetime.now()
only_date, only_time = d.date(), d.time()
only_date = only_date.strftime('%Y.%m.%d')
only_time = only_time.strftime('%H.%M')
procTS = str(only_date) + "_" + str(only_time)
dffilecount = 0
FolderName2 = FolderName.replace("'", "")

if PC == 13:
    Base = "/Volumes/My Passport for Mac/Evidence/5 - Mobile Graphic & Video Media/"
    Local = "/Users/rommell/Desktop/L - Civil Suit/RICO Album Tables/"
    External = "/Volumes/My Passport for Mac/Evidence/5 - Media Documentation/"

if PC == 14:
    Base = "/Volumes/Files/2022 MacBook Pro/LARGE FOLDERS/Civil Suit & Evidence/Civil Suit*/5 - Mobile Graphic & Video Media/11.17.24 RICO at Hip/"
    Local = "/Users/rommellmontenegro/Desktop/Civil Suit*/5 - Mobile Graphic & Video Media/RICO Album Tables/ADMIN-Working Files/GDrive Folder Lists/GDrive All Folders/"
    External = "/Volumes/Files/2022 MacBook Pro/LARGE FOLDERS/Civil Suit & Evidence/Civil Suit*/5 - Mobile Graphic & Video Media/FILES TABLES/"

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

files = []

creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret_833249459653-01pcjla0c9re30fgr2dfipcgg5lgq6j0.apps.googleusercontent.com.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

# Call the Drive v3 API
drive = build("drive", "v3", credentials=creds)

# First, get the folder ID by querying by mimeType and name
folderId = drive.files().list(q = query, pageSize=1000, fields="nextPageToken,files(id, name,webViewLink,size)").execute()

# this gives us a list of all folders with that name
folderIdResult = folderId.get('files', [])
# however, we know there is only 1 folder with that name, so we just get the id of the 1st item in the list
id = folderIdResult[0].get('id')
folderid = pd.Series(id)

# Now, using the folder ID gotten above, we get all the files from that particular folder in a dictionary file of the resulting file id, name, and weblink FOR ONLY the first 1000 results.
results = drive.files().list(q = "'" + id + "' in parents", pageSize=1000, fields="nextPageToken, files(id, name,webViewLink,size)").execute()

#Creates in a smaller dictionary file with only the data of file id, name, and weblink which can be made into a dataframe object.
items = results.get('files', [])

page_token = None


while True:
# Now, using the folder ID gotten above, we get all the files from that particular folder in a dictionary file of the resulting file id, name, and weblink results.
  response = (
      drive.files()
      .list(
          #q= "'" + id + "' in parents",
          q="mimeType = 'application/vnd.google-apps.folder'",
          spaces="drive",
          fields="nextPageToken, files(id, name, webViewLink, size)",
          pageToken=page_token,
      )
      .execute()
  )

#Creates in a smaller dictionary file with only the data of file id, name, and weblink which can be made into a dataframe object.
  for file in response.get("files", []):
    items.append(file)
  files.extend(response.get("files", []))

  page_token = response.get("nextPageToken", None)
  if page_token is None:
    break

#Creates the dataframe of the results    
df = pd.DataFrame(items)
df = df.drop_duplicates()
if 'name' not in df.columns:
    df['name'] = np.nan
    df = df[df.name != '.DS_Store']
#    df.drop(df.loc[df['name']=='.DS_Store'].index, inplace=True)
#    df.drop(df[df['name'] == '.DS_Store'].index, inplace=True)
dffilecount = str(len(df))

#df = df.sort_values(by='size')
df.reset_index(drop=True, inplace=True)

df['date-time processed'] = procTS
if 'size' not in df.columns:
    df['size'] = np.nan
df['size'] = df['size'].fillna(0)
df['size'] = df['size'].astype(int)
foldersize = df['size'].sum()
df['MB_size'] = df['size']/1000000


df['File Type'] = np.where(df['size'] == 0, 'Folder', df['name'].str[-4:])

df['File Type'].replace('docx', '.docx', inplace=True)
df['File Type'].replace('.MOV', '.mov', inplace=True)
df['File Type'].replace('xlsx', '.xlsx', inplace=True)
df['File Type'].replace('.JPG', '.jpg', inplace=True)
df['File Type'].replace('HEIC', '.heic', inplace=True)
df['File Type'].replace('PNG', '.png', inplace=True)

unique_names = df['File Type'].unique()

column_order = ['name', 'id', 'webViewLink', 'File Type', 'date-time processed', 'MB_size']
tabname = FolderName2[:30]

suffix = "_GDrive_Files.xlsx"
filename = only_date + "_" + only_time + "_" + FolderName2 + "_(" + dffilecount + ")" + suffix
HD_Dest = Local + filename

tab1 = df[:1001]
tab2 = df[1001:2001]
tab3 = df[2001:3001]
tab4 = df[3001:4001]
tab5 = df[4001:5001]
tab6 = df[5001:6001]
tab7 = df[6001:7001]
tab8 = df[7001:8001]
tab9 = df[8001:9001]
tab10 = df[9001:10001]
tab11 = df[10001:11001]
tab12 = df[11001:]
All = df

tabname1 = tabname + " 1"
tabname2 = tabname + " 2"
tabname3 = tabname + " 3"
tabname4 = tabname + " 4"
tabname5 = tabname + " 5"
tabname6 = tabname + " 6"
tabname7 = tabname + " 7"
tabname8 = tabname + " 8"
tabname9 = tabname + " 9"
tabname10 = tabname + " 10"
tabname11 = tabname + " 11"
tabname12 = tabname + " 12"
tabnameAll = " All"

with pd.ExcelWriter(HD_Dest) as writer:
    tab1[column_order].to_excel(writer, sheet_name=tabname1, index=False)
    tab2[column_order].to_excel(writer, sheet_name=tabname2, index=False)
    tab3[column_order].to_excel(writer, sheet_name=tabname3, index=False)
    tab4[column_order].to_excel(writer, sheet_name=tabname4, index=False)
    tab5[column_order].to_excel(writer, sheet_name=tabname5, index=False)
    tab6[column_order].to_excel(writer, sheet_name=tabname6, index=False)
    tab7[column_order].to_excel(writer, sheet_name=tabname7, index=False)
    tab8[column_order].to_excel(writer, sheet_name=tabname8, index=False)
    tab9[column_order].to_excel(writer, sheet_name=tabname9, index=False)
    tab10[column_order].to_excel(writer, sheet_name=tabname10, index=False)
    tab11[column_order].to_excel(writer, sheet_name=tabname11, index=False)
    tab12[column_order].to_excel(writer, sheet_name=tabname12, index=False)
    All[column_order].to_excel(writer, sheet_name=tabnameAll, index=False)    
    
suffix = "_GDrive_All_Folders_1.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix

with pd.ExcelWriter(HD_Dest) as writer:
    tab1[column_order].to_excel(writer, sheet_name=tabname1, index=False)

suffix = "_GDrive_All_Folders_2.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix
    
with pd.ExcelWriter(HD_Dest) as writer:
    tab2[column_order].to_excel(writer, sheet_name=tabname2, index=False)

suffix = "_GDrive_All_Folders_3.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix
    
with pd.ExcelWriter(HD_Dest) as writer:
    tab3[column_order].to_excel(writer, sheet_name=tabname3, index=False)

suffix = "_GDrive_All_Folders_4.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix
    
with pd.ExcelWriter(HD_Dest) as writer:
    tab4[column_order].to_excel(writer, sheet_name=tabname4, index=False)

suffix = "_GDrive_All_Folders_5.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix
    
with pd.ExcelWriter(HD_Dest) as writer:
    tab5[column_order].to_excel(writer, sheet_name=tabname5, index=False)

suffix = "_GDrive_All_Folders_6.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix
    
with pd.ExcelWriter(HD_Dest) as writer:
    tab6[column_order].to_excel(writer, sheet_name=tabname6, index=False)

suffix = "_GDrive_All_Folders_7.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix
    
with pd.ExcelWriter(HD_Dest) as writer:
    tab7[column_order].to_excel(writer, sheet_name=tabname7, index=False)

suffix = "_GDrive_All_Folders_8.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix

with pd.ExcelWriter(HD_Dest) as writer:
    tab8[column_order].to_excel(writer, sheet_name=tabname8, index=False)
    
suffix = "_GDrive_All_Folders_9.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix
    
with pd.ExcelWriter(HD_Dest) as writer:
    tab9[column_order].to_excel(writer, sheet_name=tabname9, index=False)

suffix = "_GDrive_All_Folders_10.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix
    
with pd.ExcelWriter(HD_Dest) as writer:
    tab10[column_order].to_excel(writer, sheet_name=tabname10, index=False)

suffix = "_GDrive_All_Folders_11.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix
    
with pd.ExcelWriter(HD_Dest) as writer:
    tab11[column_order].to_excel(writer, sheet_name=tabname11, index=False)

suffix = "_GDrive_All_Folders_12.xlsx"
HD_Dest = Local + only_date + "_" + only_time + "_(" + dffilecount + ")" + suffix

with pd.ExcelWriter(HD_Dest) as writer:
    tab12[column_order].to_excel(writer, sheet_name=tabname12, index=False)

/var/folders/5k/m2kjbr8969j9g5j1fq8vnxkm0000gn/T/ipykernel_52219/4097022071.py:138: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['File Type'].replace('docx', '.docx', inplace=True)
/var/folders/5k/m2kjbr8969j9g5j1fq8vnxkm0000gn/T/ipykernel_52219/4097022071.py:139: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v